In [1]:
import os
import pandas as pd

In [2]:
# Function to generate segments for a single video
def generate_segments(row, segment_duration, overlap_duration):
    segments = []
    start = 0
    while start + segment_duration <= row['duration']:
        end = start + segment_duration
        narr_sec = (start + end) / 2
        start_frame = int(round(start * row['fps']))
        end_frame = int(round(end * row['fps']))
        segments.append({
            'video_id': row['video_id'],
            'start_sec': start,
            'stop_sec': end,
            'narration_sec': narr_sec,
            'start_frame': start_frame,
            'stop_frame': end_frame
        })

        start += segment_duration - overlap_duration
    output = pd.DataFrame(segments)
    output['narration_sec'] = (output['start_sec'] + output['stop_sec']) / 2
    return output

def assign_narration_id(group):
    group['narration_id'] = group['video_id'] + '_' + (group.index + 1).astype(str)
    return group

In [3]:
# Define segment and overlap durations
segment_duration = 1.06667  # seconds
overlap_duration = 0.53333  # seconds

In [4]:
output_dir = "/private/home/arjunrs1/AudioGround/annotations/EPIC"
output_filename = "EPIC_climer_feature_times.pkl"

metadata_path = "/private/home/arjunrs1/CliMer/data/epic/EPIC_metadata.csv"
metadata = pd.read_csv(metadata_path)

In [5]:
# Apply the function to each row in the DataFrame
segments_df = pd.concat([generate_segments(row, segment_duration, overlap_duration) for index, row in metadata.iterrows()], ignore_index=True)
segments_df = segments_df.groupby('video_id').apply(assign_narration_id).reset_index(drop=True)
segments_df = segments_df.set_index('narration_id')

/tmp/ipykernel_1352817/1040196744.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  segments_df = segments_df.groupby('video_id').apply(assign_narration_id).reset_index(drop=True)


In [6]:
output_filepath = os.path.join(output_dir, output_filename)
os.makedirs(output_dir, exist_ok=True)
segments_df.to_pickle(output_filepath)
print(f"Data saved to {output_filepath}")

Data saved to /private/home/arjunrs1/AudioGround/annotations/EPIC/EPIC_climer_feature_times.pkl


In [7]:
segments_df

,video_id,start_sec,stop_sec,narration_sec,start_frame,stop_frame
narration_id,,,,,,
P01_01_1,P01_01,0.00000,1.06667,0.533335,0,64
P01_01_2,P01_01,0.53334,1.60001,1.066675,32,96
P01_01_3,P01_01,1.06668,2.13335,1.600015,64,128
P01_01_4,P01_01,1.60002,2.66669,2.133355,96,160
P01_01_5,P01_01,2.13336,3.20003,2.666695,128,192
...,...,...,...,...,...,...
P37_103_674122,P37_103,391.47156,392.53823,392.004895,19574,19627
P37_103_674123,P37_103,392.00490,393.07157,392.538235,19600,19654
P37_103_674124,P37_103,392.53824,393.60491,393.071575,19627,19680
